# Input Description

Job  Machine
3    5
Job# Machine#   M1     M2     M3            VG
1    3          1  10  2  30  3  20         1
1    4          1  12  5  13  3  21  4  28  2
1    3          1  21  2  10  3  11         3

3 5
1 3 1 10 2 30 3 20 1
1 4 1 12 5 13 3 21 4 28 2
1 3 1 21 2 10 3 11 3

# Data Parsing

In [80]:
def readFile(filePath):
    with open(filePath) as fp:
        line = fp.readline()
        nm = line.strip().split()
        machineCnt = nm[1]
        jobs = []
        machines = []
        processing_time = []
        VG = []

        for i in range(int(nm[0])):
            line = fp.readline()
            cv = list(map(int, line.split()))
            operations = []
            times = []
            j = 1
            while j < len(cv):
                # 각 op 별 machine, processing_time 할당
                k = cv[j]
                operation = []
                machine_l = []
                for kj in range(k):
                    j = j + 1
                    if (j >= len(cv)):
                        break;
                    # machine 할당
                    machine = cv[j]
                    j = j + 1
                    if (j >= len(cv)):
                        continue;
                    # processing_time 할당
                    processing_t = cv[j]
                    machine_l.append(machine)

                    processing_time.append(processing_t)
                    operation.append({'machine': machine, 'processingTime': processing_t})
                j = j + 1
                if len(operation) > 0: operations.append(operation)
                if len(machine_l) > 0: machines.append(machine_l)

            
            # VG 할당
            if (j > len(cv)):
                vgcnt = -1
                VG_L = [[] for i in range(cv[0])]
                for v in range(cv[0]):
                    vgcnt -= v
                    VG_L[v].append(int(cv[vgcnt]))
                
                VG_L.reverse()
                VG.append(VG_L)


            if len(operations) > 1:
                for o in range(len(operations)):
                    jobs.append([operations[o]])
            else:
                jobs.append(operations)
    return jobs, VG, machines, processing_time, {'machineCnt': int(machineCnt), 'jobs': jobs}


In [81]:
jobs, VG, machines, processing_time, jobDicts = readFile("./test_data.txt")

[[1, 2, 3]]
[[1, 2, 3], [1, 5, 3, 4], [1, 2, 3]]
[[1, 2, 3], [1, 5, 3, 4], [1, 2, 3], [1, 2, 3]]
[[1, 2, 3], [1, 5, 3, 4], [1, 2, 3], [1, 2, 3], [1, 2, 3]]
[[1, 2, 3], [1, 5, 3, 4], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 5, 3, 4]]
[[1, 2, 3], [1, 5, 3, 4], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 5, 3, 4], [1, 2, 3]]


# Population Generation

In [83]:
import random

def generateN(jobDicts, VG):
    if len(VG) > 0:
        N = [[]]
        jobs = jobDicts['jobs']
        i = 0
        for v in VG:
            cnt = 0
            for j in v:
                cnt += 1
                if cnt > 1:
                    N.append([])
                    N[cnt-1].append(i)
                else:
                    N[cnt-1].append(i)
            i += 1
        for i in N:
            random.shuffle(i)
    else:
        N = []

        jobs = jobDicts['jobs']
        i = 0
        for job in jobs:
            for op in job:
                N.append(i)
            i = i+1
        random.shuffle(N)
    return N

In [190]:
from itertools import permutations
def perm_table(VG, machines):
    VG_tables = []
    perm = []
    cnt = 0
    for j in VG:
        if len(j) > 1:
            for k in j:
                perm.append(k[0])
        else:
            perm.append(j[0][0])

    for i in perm:
        per_list = []
        for k in permutations(machines[cnt], i):
            if(len(machines[cnt]) == i) or i==1:
                continue;
            per_list.append(list(k))
        if(len(machines[cnt]) == i) or i==1:
            per_list[:0] = machines[cnt]
        VG_tables.append(per_list)
        cnt += 1
    VG_tables
    return VG_tables


In [191]:
perm_table = perm_table(VG, machines) # [1, 2, 3]

In [200]:
def generateM(jobDicts, VG):
    M = [[]]
    if(len(VG) > 0):
        cnt = 0
        for v in VG:
            j = 0
            for k in v:
                if j > 0: 
                    M.append([])
                    cnt += 1
                if(len(perm_table[cnt]) > v[j][0]):
                    random.shuffle(perm_table[cnt])
                    if(type(perm_table[cnt][0]) != int):
                        M[j] = M[j]  + perm_table[cnt][0]
                    else:
                        M[j].append(perm_table[cnt][0])
                else:
                    if(type(perm_table[cnt]) != int):
                        M[j] = M[j]  + perm_table[cnt]
                    else:
                        M[j].append(perm_table[cnt])
                j += 1
            cnt += 1
    else:
        jobs = jobDicts['jobs']
        cnt = 0
        for job in jobs:
            for op in job:
                cnt += 1
                randomMachine = random.randint(0, len(op)-1)
                M.append(randomMachine)
    return M

In [201]:
def initial_generation(jobDicts, VG):
    pop_gen = []
    for i in range(400):
        NS = generateN(jobDicts, VG)
        MS = generateM(jobDicts, VG)
        pop_gen.append((NS, MS))

    return pop_gen

In [202]:
parameter = initial_generation(jobDicts,VG) # NS MS
parameter

# Calculate Makespan

In [205]:
(n,m) = ([[3, 0, 5, 2, 4, 1], [1]], [[2, 5, 1, 1, 2, 3, 1, 3, 4, 1, 2, 3], [1]])
job_lists = []
current = 0
if(len(VG) > 0):
    for v in VG:
        append_l = []
        for i in v:
            append_l.append(m[current])
            current += 1
        job_lists.append(append_l)
print(job_lists)
#         for i in range(v):
#             append_l.append(m[current])
#             current += 1
#         job_lists.append(append_l)

IndexError: list index out of range

In [24]:
# VG 별 machine 나누기
def split_by_machine(VG, param, jobDicts):
    (n,m) = param
    job_lists = []
    current = 0
    if(len(VG) > 0):
        for v in VG:
            append_l = []
            for i in range(v):
                append_l.append(m[current])
                current += 1
            job_lists.append(append_l)
    else:
        current = 0
        for index, job in enumerate(jobDicts['jobs']):
            job_lists.append(m[current:current+len(job)])
            current += len(job)

    return job_lists

In [25]:
def is_free(tab, start, duration):
    for k in range(start, start+duration):
        if not tab[k]:
            return False
    return True

def find_first_proc_time(start_ctr, duration, machine_jobs):
    max_duration_list = []
    max_duration = start_ctr + duration

    if machine_jobs:
        for job in machine_jobs:
            max_duration_list.append(job[2] + job[1]) 

        max_duration = max(max(max_duration_list), start_ctr) + duration

    machine_used = [True] * max_duration

    for job in machine_jobs:
        start = job[2]
        long = job[1]
        for k in range(start, start + long):
            machine_used[k] = False

    for k in range(start_ctr, len(machine_used)):
        if is_free(machine_used, k, duration):
            return k

In [26]:
def calculateMakespan(cpl, jobDicts):
    times_taken = getMakespans(cpl, jobDicts)
    
    max_per_machine = []
    for machine in times_taken:
        max_d = 0
        for job in machine:
            end = job[2]
            if end > max_d:
                max_d = end
        max_per_machine.append(max_d)
    return max(max_per_machine)

In [27]:
def getMakespans(cpl, jobDicts):
    o = jobDicts['jobs']
    (n, m) = cpl
    
    cnt = 0
    ops = [[] for i in range(jobDicts['machineCnt'])]
    time_taken = [[] for i in range(jobDicts['machineCnt'])]
    m_splits = split_by_machine(VG, (n, m), jobDicts)
    outer_shell = [0] * len(m_splits)
    time_outer_shell = [0] * len(m_splits)
    n_vg = []
    print("n : ", n)
    print("VG : ", VG)
    for i in n:  
        for j in range(VG[i]):
            print(i, j) 
            n_vg.append(i)
     
    for i in n_vg:
        idx_m = m_splits[i][outer_shell[i]]
        
        idx_m = idx_m-1
        
        if (len(VG) > 0):
            machine = [i['machine'] for i in o[i][0] if i['machine']==idx_m+1][0]
            proc_t = [i['processingTime'] for i in o[i][0] if i['machine']==idx_m+1][0]
            
        else: 
            machine = o[i][outer_shell[i]][idx_m]['machine']
            proc_t = o[i][outer_shell[i]][idx_m]['processingTime']
        start_t = time_outer_shell[i]

            
        tag = "{}".format(i) # , outer_shell[i]+1)
        outer_shell[i] += 1
        if(len(time_taken[machine-1])>0):
            time = (start_t + proc_t)
            lng = len(time_taken[machine-1])-1
            time = time_taken[machine-1][lng] + time
            time_taken[machine-1].append(time)
            ops[machine-1].append((tag, time_taken[machine-1][lng], time))
        else:
            time_taken[machine-1].append((start_t + proc_t))
            ops[machine-1].append((tag, start_t, (start_t + proc_t)))
    
    # calc = calculateMakespan(ops)
    return ops




In [28]:
sortedPop = sorted(parameter, key=lambda cpl: calculateMakespan(cpl, jobDicts))

# minimum makespan population
# sortedPop[0][0], sortedPop[0][1]

TypeError: 'list' object cannot be interpreted as an integer

# Draw Gantt Chart

In [31]:
gantt = getMakespans((sortedPop[0][0], sortedPop[0][1]), jobDicts)

NameError: name 'sortedPop' is not defined

In [92]:
# gantt
# m_cnt = jobDicts['machineCnt']
import datetime
import matplotlib.pyplot as plt
colorbox = ['yellow', 'whitesmoke', 'lightyellow',
            'khaki', 'silver', 'pink', 'lightgreen', 'orange', 'grey', '#8ca8df', 'brown']

for i in range(100):
    colorArr = ['1', '2', '3', '4', '5', '6', '7',
                '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0, 14)]
    colorbox.append("#" + color)

zzl = plt.figure(figsize=(12, 4))

mcnt = 0
for i in gantt:
    for j in i:
        mText = mcnt + 1.5
        mPoint1 = j[1]
        mPoint2 = j[2]
        Word = j[0]
        x1 = mPoint1
        y1 = mText - 0.8
        x2 = mPoint2
        y2 = mText - 0.8
        x3 = mPoint2
        y3 = mText
        x4 = mPoint1
        y4 = mText
        
        color_per_job = int(j[0][:j[0].find('-')])
        plt.fill([x1, x2, x3, x4], [y1, y2, y3, y4],
                 color=colorbox[color_per_job])
        plt.text(0.5 * mPoint1 + 0.5 * mPoint2 - 3, mText - 0.5, Word)
        
        
    mcnt += 1
        
        
plt.xlabel('Time')
plt.ylabel('Machine')
plt.tight_layout()
plt.show()

ValueError: invalid literal for int() with base 10: ''

<Figure size 864x288 with 0 Axes>